# Data extraction

If this is being run locally and not on the kaggle platform, please run `data_download.sh`.

In [175]:
from os import walk, listdir
from os.path import isfile, join

# this will only work after deleting the zip file.
for (dirpath, dirnames, filenames) in walk('./data'):
    patients = dirnames
    break
print (patients)
    
files = []
for i in patients:
    for j in ['/0/', '/1/']:
        path = './data/' + i + j
        these_files = [(path+f) for f in listdir(path) if isfile(join(path, f))]
        files += these_files

['10253', '10254', '10255']


In [176]:
import numpy as np
from matplotlib.image import imread

In [177]:
imgs_0 = []
imgs_1 = []
for i in files:
    if int(i[-5]) == 0:
        imgs_0.append(imread(i))
    else:
        imgs_1.append(imread(i))
imgs_0 = np.array(imgs_0)
imgs_1 = np.array(imgs_1)

In [178]:
imgs_0.shape, imgs_1.shape

((1432, 50, 50, 3), (237, 50, 50, 3))

## On data selection
This dataset is very unbalanced. Efforts are going to be made to:
- idc vs. non-idc is a 50/50 split
- train vs. test is a 65/35 split

### 50/50 split of IDC vs Non-IDC

In [179]:
non_idc_n = imgs_0.shape[0]
n_idc_n = imgs_1.shape[0]
non_idc_to_delete = abs(imgs_0.shape[0] - imgs_1.shape[0])
# not inspected I know, but the dataset shows more non-idc than idc

In [180]:
def rand_indices(n_samples_to_delete):
    # generates unique indexes
    rand_indices = []
    while len(rand_indices) < n_samples_to_delete:
        sample = np.random.choice(n_samples_to_delete)
        if sample not in rand_indices:
            rand_indices.append(sample)
    return np.array(rand_indices).copy()  # something was happening so I just hacked it

In [181]:
idx = rand_indices(non_idc_to_delete)
imgs_0 = np.delete(imgs_0, idx, axis=0)
imgs_0.shape

(237, 50, 50, 3)

### 65/35 split of training and test

In [182]:
# shuffle and split
# shuffle
np.random.shuffle(imgs_0)
np.random.shuffle(imgs_1)

# split
split_idx = int(np.floor(0.65 * imgs_0.shape[0]))  # idc and non idc share the same size

train_0 = imgs_0[:split_idx]
train_1 = imgs_1[:split_idx]

test_0 = imgs_0[split_idx:]
test_1 = imgs_1[split_idx:]

In [183]:
(train_0.shape, test_0.shape, train_1.shape, test_1.shape)

((154, 50, 50, 3), (83, 50, 50, 3), (154, 50, 50, 3), (83, 50, 50, 3))

Cool! We have the dataset prepared (do not read 'cleaned') and wrapped in numpy arrays. Now we only have to create one-hot encodings.

### Formatting for `tensorflow`

In [184]:
train_0_labels = np.zeros(train_0.shape[0])
test_0_labels = np.zeros(test_0.shape[0])

train_1_labels = np.ones(train_1.shape[0])
test_1_labels = np.ones(test_1.shape[0])

train = np.append(train_0, train_1, axis=0)
train_labels = np.append(train_0_labels, train_1_labels, axis=0)

test = np.append(test_0, test_1, axis=0)
test_labels = np.append(test_0_labels, test_1_labels, axis=0)

from sklearn.utils import shuffle
train, train_labels = shuffle(train, train_labels)
test, test_labels = shuffle(test, test_labels)

def get_one_hot(targets, nb_classes):
    # https://stackoverflow.com/a/42874726
    return np.eye(nb_classes)[np.array(targets.astype(np.int32)).reshape(-1)]

train_labels = get_one_hot(train_labels, 2)
test_labels = get_one_hot(test_labels, 2)

In [185]:
import pickle
with open('dataset.npys', 'wb') as handle:
    pickle.dump((train, train_labels, test, test_labels), handle)